# Setup

## Instructions

This notebook utilizes the Kaggle API. Follow these steps in order to get the necessary credentials to continue:

1. Sign up for a Kaggle account at https://www.kaggle.com.
2. Go to the 'Account' tab of your user profile - ```https://www.kaggle.com/{username}/account```.
3. Select 'Create New API Token' under 'API' section.
    - This will trigger the download of kaggle.json, a file containing your API credentials. 
4. Place this file in the location:
    - ~/.kaggle/kaggle.json (for macOS/unix)
    - C:/Users/username/.kaggle/kaggle.json (for Windows) 
    - You can check the exact location, sans drive, with echo %HOMEPATH%). 
    - You can define a shell environment variable KAGGLE_CONFIG_DIR to change this location to:
        - $KAGGLE_CONFIG_DIR/kaggle.json (for macOS/unix)
        - %KAGGLE_CONFIG_DIR%\kaggle.json (for Windows)

Note: this notebook uses functions written in Python to query the Kaggle API. **This code will only work for python 3.7 or later**.

## Additional Information

Documentation Guide:
- Kaggle API ([Kaggle](https://www.kaggle.com/docs/api))
- Kaggle API ([GitHub](https://github.com/Kaggle/kaggle-api)) 

## Overview of workflow

#### Import necessary libraries

#### Query API based on search terms and search types

Define functions to query API based on search terms and search types:
1.	Function `get_search_output` queries the Kaggle API with the specified search term (e.g., “machine learning”) and search type (must be either “datasets” or “kernels”)
    - Searches across all returned pages
    - Calls function `_convert_string_csv_output_to_dataframe` clean up results (which include X)
    
    
2.	Function `_convert_string_csv_output_to_dataframe` converts results from API (strings in semi-structured table) to dataframe format


3.	Function `get_search_outputs` queries the Kaggle API for all combinations of search terms and search types specified and returns the results as a dictionary of dataframes (one dataframe for each query combination)
    - Calls function `get_search_output`

Run functions for specified search terms and search types


#### Query API for full metadata for hits from initial query

Following the initial query to return high level metadata for hits of initial query, define functions to retrieve full metadata associated with each object

4.	Function `_retrieve_object_json` uses the path for each object (e.g., <example>) to query the API for the associated metadata
    
    
5.	Function `get_metadata` extracts metadata associated with each object and formats as dataframe
    - Calls function `retrieve_object_json`
    - <need clarification on section “perform metadata extraction” – this calls `get_metadata` and puts results in a dictionary?>

Run functions to pull metadata for each object returned by first query.

We now have a dictionary of results from the first API query (which includes XYZ) and a dictionary of results with metadata for each object, based on path from first query.

#### Merge results
6.	Function `merge_search_and_metadata_dicts` merges these two dictionaries and (optional) saves the resulting dataframe to csv
    - <need clarification on if `left_on` and `right_on` always need to be specified, in which case, can this be an argument in the function itself, to streamline>
    - <need clarification on output format – can save dataframe as csv, but output is ordered dict – why not dataframe?>

Run merge function to output a csv (??) of results.

## Import libraries

In [1]:
# Import kaggle, installing if necessary
try:
    import kaggle
except ImportError as e:
    !pip3 install kaggle
    import kaggle
    
import subprocess # Used to run unix commands
import pandas as pd # For storing/manipulating command data
from io import StringIO # Lets us read csv string output from command into DataFrame
import json # Reading back the metadata files
from tqdm import tqdm # Gives status bar on loop completion
import itertools # For efficient looping over queries
import os # Exporting saved results
from collections import OrderedDict
from utils import flatten_nested_df

## Query API based on search terms and search types

In [2]:
def get_search_outputs(search_terms, search_types, flatten=False):
    """
    Call the Kaggle API for each search term and search type. 
    Results are retured in results['{term}_{type}'] = df
    
    Params:
    - search_terms (list-like): collection of search terms to query over
    - search_types (list-like): collection of search types to query over
    - flatten (bool): optional (default=False)
    
    Returns:
    - results (dict): dictionary consisting of returned DataFrames from get_search_output for each query
    """
    
    num_searches = len(search_terms) * len(search_types)
    results = OrderedDict()
    
    for search_term, search_type in itertools.product(search_terms, search_types):
        results[(search_term, search_type)] = get_search_output(search_term, search_type)
        
    return results

In [3]:
def get_search_output(search_term, search_type, flatten=False):
    """
    Calls the Kaggle API with the specified search term and returns the search output results.
    
    Params:
    - search_term (str): keyword to seach for
    - search_type (str): objects to search over (must be either datasets or kernels)
    - flatten (bool): optional (default=False)
    
    Returns:
    - df (pandas.DataFrame): DataFrame containing the output of the search query
    """
    
    # Make sure our input is valid
    assert isinstance(search_type, str), 'Search term must be a string'
    assert search_type in ('datasets', 'kernels'), 'Search can only be conducted over datasets or kernels'
    
    # Set search variables
    page_idx = 1
    search_output = ''
    cumulative_output = ''
    completion_phrase = f'No {search_type} found\n'
    
    # Pulls the records for a single page of datasets for the given search term
    # Runs the command, captures the output in stdout, reads it from stdout, and decodes it to str from binary
    search_output = subprocess.run(['kaggle', search_type, 'list', '-v',
                                     '-s', f'"{search_term}"', 
                                     '-p', str(page_idx)], 
                                    capture_output=True).stdout.decode()

    # Once we no longer see new output, we stop
    while search_output != completion_phrase:
        # Accumulate the output
        cumulative_output = cumulative_output + search_output

        # Increments the page count for searching
        page_idx += 1

        # Pulls the records for a single page of datasets for the given search term
        # Runs the command, captures the output in stdout, reads it from stdout, and decodes it to str from binary
        search_output = subprocess.run(['kaggle', search_type, 'list', '-v',
                                         '-s', f'"{search_term}"', 
                                         '-p', str(page_idx)], 
                                        capture_output=True).stdout.decode()

        # Remove header row
        if search_output != completion_phrase:
            search_output = '\r\n'.join(search_output.split('\r\n')[1::])
    
    # Convert search output to DataFrame
    search_df = _convert_string_csv_output_to_dataframe(cumulative_output)
    
    # Rename columns to match names present in metadata df
    search_df.rename(columns={'ref': 'id', 
                              'downloadCount': 'totalDownloads', 
                              'voteCount': 'totalVotes'}, 
                     inplace=True)
        
    if flatten:
        search_df = flatten_nested_df(search_df)
    
    return search_df

In [4]:
def _convert_string_csv_output_to_dataframe(output):
    """
    Given a string variable in csv format, returns a Pandas DataFrame
    
    Params:
    - output (str): csv-styled string to be converted
    
    Returns:
    - df (pandas.DataFrame): DataFrame consisting of data from 'output' string variable
    """
    
    # Create DataFrame of results
    output = StringIO(output)
    df = pd.read_csv(output)
    
    return df

#### Run initial API query functions

In [5]:
search_terms = ['korea']
search_types = ['datasets', 'kernels']

In [6]:
search_output_dict = get_search_outputs(search_terms, search_types)

In [7]:
sample_key = (search_terms[0], search_types[0])
sample_df = search_output_dict[sample_key]

In [8]:
sample_df.head()

,id,title,size,lastUpdated,totalDownloads,totalVotes,usabilityRating
0,kimjihoo/coronavirusdataset,[NeurIPS 2020] Data Science for COVID-19 (DS4C),7MB,2020-07-13 14:07:31,83975,1469,1.000000
1,bappekim/air-pollution-in-seoul,Air Pollution in Seoul,20MB,2020-04-03 16:33:49,9920,311,1.000000
2,bappekim/south-korea-visitors,South Korea Visitors,99KB,2020-06-04 08:53:36,1048,24,1.000000
3,hongsean/korea-income-and-welfare,Korea Income and Welfare,772KB,2020-12-20 13:05:27,590,15,0.970588
4,bryanpark/korean-single-speaker-speech-dataset,Korean Single Speaker Speech Dataset,3GB,2020-03-15 08:56:42,5583,108,0.750000


## Query API for full metadata for hits from initial query

Note: Unable to find a way to store metadata in memory as opposed to saving file, but this workaround appears to be functional.

In [9]:
def _retrieve_object_json(object_path):
    """
    Queries Kaggle for metadata json file & returns the json data as a dictionary
    
    Params:
    - object_path (str): path for the dataset
    
    Returns:
    - metadata_dict (dict): dictionary containing json metadata
    """
    
    # Download the metadata
    subprocess.run(['kaggle', 'datasets', 'metadata', object_path])

    # Access the metadata and load it in as a dictionary
    with open('dataset-metadata.json') as file:
        json_data = json.load(file)
        
    return json_data

In [10]:
def get_metadata(object_paths, flatten=False):
    """
    Retrieves the metadata for the file/files listed in object_paths
    
    Params:
    - object_paths (str/list-like): string or list of strings containing the paths for the objects
    - flatten (bool): optional (default=False)
    
    Returns:
    - metadata_df (pandas.DataFrame): DataFrame containing metadata for the requested datasets
    """
    
    # If a singular search term is provided as a string, need to wrap it in a list
    if type(object_paths) == str:
        object_paths = [object_paths]
    
    # Make sure our input is valid
    assert len(object_paths) > 0, 'Please enter at least one object id'
        
    # Run first query
    json_data = _retrieve_object_json(object_paths[0])
        
    # Create DataFrame to store metadata in, using columns found in first query, and then add query info
    metadata_df = pd.DataFrame(columns=json_data.keys(), dtype=object)
    metadata_df = metadata_df.append(json_data, ignore_index=True)
        
    # Pulls metadata information for each dataset found above
    for object_path in tqdm(object_paths[1::]):
        # Download & load the metadata
        json_data = _retrieve_object_json(object_path)

        # Store the metadata into our DataFrame created above
        metadata_df = metadata_df.append(json_data, ignore_index=True)
    
    if flatten:
        metadata_df = flatten_nested_df(metadata_df)
        
    return metadata_df

#### Run metadata extraction functions

In [11]:
## Extract IDs from DataFrame, and returns as list of strings
metadata_dict = OrderedDict()

for query, df in search_output_dict.items():
    print(f'Retrieving {query} metadata')
    # Create object paths
    _, search_type = query
    object_paths = df.id.values
    
    metadata_dict[query] = get_metadata(object_paths, flatten=True)

Retrieving ('korea', 'datasets') metadata


100%|██████████| 178/178 [07:18<00:00,  2.46s/it]


Retrieving ('korea', 'kernels') metadata


100%|██████████| 1019/1019 [1:08:01<00:00,  4.01s/it]


In [23]:
metadata_dict[('korea', 'datasets')]

,id,id_no,datasetId,datasetSlug,ownerUser,usabilityRating,totalViews,totalVotes,totalDownloads,title,...,collaborators_12_role,collaborators_12_username,collaborators_13_role,collaborators_13_username,collaborators_14_role,collaborators_14_username,collaborators_15_role,collaborators_15_username,collaborators_16_role,collaborators_16_username
0,kimjihoo/coronavirusdataset,527325,527325,coronavirusdataset,kimjihoo,1.000000,491478,1469,83975,[NeurIPS 2020] Data Science for COVID-19 (DS4C),...,writer,parkjuhwan,writer,leewoncheol,writer,younajung,writer,minty99,writer,byeongukyu
1,bappekim/air-pollution-in-seoul,576393,576393,air-pollution-in-seoul,bappekim,1.000000,81401,311,9920,Air Pollution in Seoul,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bappekim/south-korea-visitors,692628,692628,south-korea-visitors,bappekim,1.000000,5879,24,1048,South Korea Visitors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hongsean/korea-income-and-welfare,1046735,1046735,korea-income-and-welfare,hongsean,0.970588,3060,15,590,Korea Income and Welfare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bryanpark/korean-single-speaker-speech-dataset,19829,19829,korean-single-speaker-speech-dataset,bryanpark,0.750000,83614,108,5583,Korean Single Speaker Speech Dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,glhr00/rgbt-driveability-segmentation-kaist,1340078,1340078,rgbt-driveability-segmentation-kaist,glhr00,0.562500,46,0,1,RGB-T driveability segmentation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,llkdev/gender-data,231235,231235,gender-data,llkdev,0.125000,341,0,8,Gender Data,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,chenykfrank/immc21,1135726,1135726,immc21,chenykfrank,0.117647,45,0,2,immc21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,gremmn/gas-prices,1108372,1108372,gas-prices,gremmn,0.117647,171,0,16,gas prices,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Combine results of initial and metadata queries

In [12]:
def merge_search_and_metadata_dicts(search_dict, metadata_dict, on, save=False):
    """
    Merges together all of the search and metadata DataFrames by the given 'on' key
    
    Params:
    - search_dict (dict): dictionary of search output results
    - metadata_dict (dict): dictionary of metadata results
    - on (str/list-like): column name(s) to merge the two dicts on
    - save=False, optional (bool/list-like): specifies if the output DataFrames should be saved
        If True: saves to file of format 'data/kaggle/kaggle_{search_term}_{search_type}.csv'
        If list-like: saves to respective location in list of save locations
            Must contain enough strings (one per query; len(search_terms) * len(search_types))
            
    Returns:
    - df_dict (OrderedDict): OrderedDict containing all of the merged search/metadata dicts
    """

    # Make sure the dictionaries contain the same searches
    assert search_dict.keys() == metadata_dict.keys(), 'Dictionaries must contain the same searches'
    
    num_dataframes = len(search_dict)
    
    # Ensure the save variable data is proper
    try:
        if isinstance(save, bool):
            save = [save] * num_dataframes
        assert len(save) == num_dataframes
    except:
        raise ValueError('Incorrect save value(s)')

    # Ensure the on variable is proper
    try:
        assert len(on) == 2 or isinstance(on, str)
        if (len(on) == 2) and (not isinstance(on, str)):
            left_on, right_on = on
            on = None
    except:
        raise ValueError('Incorrect value of "on" passed')
        
    # Merge the DataFrames
    df_dict = OrderedDict()
    for (query_key, search_df), (query_key, metadata_df), save_loc in zip(search_dict.items(), 
                                                                          metadata_dict.items(), 
                                                                          save):

        #Merge small version of "full" dataframe with "detailed" dataframe
        if on: # only one value to merge on
            df_all = pd.merge(search_df, metadata_df, on=on, how='inner')
        else:
            df_all = pd.merge(search_df, metadata_df, left_on=left_on, right_on=right_on)
            
        # Save DataFrame
        if save_loc:
            data_dir = os.path.join('data', 'figshare')
            if isinstance(save_loc, str):
                output_file = save_loc
            elif isinstance(save_loc, bool):
                # Ensure kaggle directory is already created
                if not os.path.isdir(data_dir):
                    os.path.mkdir(data_dir)
                
                search_term, search_type = query_key
                output_file = f'{search_term}_{search_type}.csv'
            else:
                raise ValueError('Save type must be bool or str')

            search_df.to_csv(os.path.join(data_dir, output_file), index=False)
        
        df_dict[query_key] = df_all
    
    return df_dict

#### need to clarify left_on, right_on and if can be incorporated into function

In [13]:
left_on = ['ref', 'title', 'usabilityRating', 'downloadCount', 'title']
right_on = ['id', 'title', 'usabilityRating', 'totalDownloads', 'title']
df_dict = merge_search_and_metadata_dicts(search_output_dict, metadata_dict, on='id')

In [14]:
output_df = search_output_dict[('korea', 'datasets')]
metadata_df = metadata_dict[('korea', 'datasets')]

#### specify differences between next two datasets - looks like initial search df then full metadata search df (result of merge function)

In [15]:
output_df.head()

,id,title,size,lastUpdated,totalDownloads,totalVotes,usabilityRating
0,kimjihoo/coronavirusdataset,[NeurIPS 2020] Data Science for COVID-19 (DS4C),7MB,2020-07-13 14:07:31,83975,1469,1.000000
1,bappekim/air-pollution-in-seoul,Air Pollution in Seoul,20MB,2020-04-03 16:33:49,9920,311,1.000000
2,bappekim/south-korea-visitors,South Korea Visitors,99KB,2020-06-04 08:53:36,1048,24,1.000000
3,hongsean/korea-income-and-welfare,Korea Income and Welfare,772KB,2020-12-20 13:05:27,590,15,0.970588
4,bryanpark/korean-single-speaker-speech-dataset,Korean Single Speaker Speech Dataset,3GB,2020-03-15 08:56:42,5583,108,0.750000


In [16]:
metadata_df.head()

,id,id_no,datasetId,datasetSlug,ownerUser,usabilityRating,totalViews,totalVotes,totalDownloads,title,...,collaborators_12_role,collaborators_12_username,collaborators_13_role,collaborators_13_username,collaborators_14_role,collaborators_14_username,collaborators_15_role,collaborators_15_username,collaborators_16_role,collaborators_16_username
0,kimjihoo/coronavirusdataset,527325,527325,coronavirusdataset,kimjihoo,1.000000,491478,1469,83975,[NeurIPS 2020] Data Science for COVID-19 (DS4C),...,writer,parkjuhwan,writer,leewoncheol,writer,younajung,writer,minty99,writer,byeongukyu
1,bappekim/air-pollution-in-seoul,576393,576393,air-pollution-in-seoul,bappekim,1.000000,81401,311,9920,Air Pollution in Seoul,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bappekim/south-korea-visitors,692628,692628,south-korea-visitors,bappekim,1.000000,5879,24,1048,South Korea Visitors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hongsean/korea-income-and-welfare,1046735,1046735,korea-income-and-welfare,hongsean,0.970588,3060,15,590,Korea Income and Welfare,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bryanpark/korean-single-speaker-speech-dataset,19829,19829,korean-single-speaker-speech-dataset,bryanpark,0.750000,83614,108,5583,Korean Single Speaker Speech Dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
